# Explorative Data Analysis in Action: global power plant distribution

In the subsequent notebooks we apply a variety of EDA techniques on a real world data set.

The data set [__Global Power-Plants__](https://www.kaggle.com/ramjasmaurya/global-powerplants) is avaiable on [Kaggle](https://www.kaggle.com/).



It was already downloaded for you and is found in the `datasets` folder:

    ../data/powerplants.csv

<img src="./_img/power_plant.webp"> 

Source: [The Quint](https://www.thequint.com/news/environment/thermal-power-plants-use-up-more-water-than-permitted-rti-data-shows/).

### Content

This dataset consists of information about power plants worldwide. Each record includes the name, country, energy source type, geographic location, start date and other data elements. In this data analysis we want to learn someting about the geospatial distribution and the energy share of power plants in the world and in specific regions. Referring to the global goals of reducing the greenhouse gas emissions the energy production sector is one of the most important one. Actual knowledge about the specific energy share of green / fossil energy source and its change in time is therefore an important information for political decissions. 



# Data preparation

**Import statements**

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = [20,9]

![](./_img/Time_data_science.png)

Source: [Gil Press (2016)](https://www.forbes.com/sites/gilpress/2016/03/23/data-preparation-most-time-consuming-least-enjoyable-data-science-task-survey-says/#55852a146f63)

## Reading the Dataset

In [ ]:
powerplants = pd.read_csv("../data/powerplants.csv")

In [ ]:
powerplants.shape

In [ ]:
powerplants.sample(10)

## Data Cleaning

[Data cleansing or data cleaning](https://en.wikipedia.org/wiki/Data_cleansing) is the process of **detecting and correcting (or removing) corrupt or inaccurate records** from a record set, table, or database and refers to **identifying incomplete, incorrect, inaccurate or irrelevant parts of the data and then replacing, modifying, or deleting the dirty or coarse data**.

### Dealing with incomplete (`NaN`) and irrelevant data

Missing values in data sets are a well-known problem as nearly everywhere, where data is measured and recorded, issues with missing values occur. Various reasons lead to missing values: values may not be measured, values may be measured but get lost or values may be measured but are considered unusable. Missing values can lead to problems, because often further data processing and analysis steps rely on complete data sets. Therefore missing values need to be replaced with reasonable values. In statistics this process is called **imputation**.

When faced with the problem of missing values it is important to understand the mechanism that causes missing data. Such an understanding is useful, as it may be employed as background knowledge for selecting an appropriate imputation strategy. 

**Check for `NaN`**

Note that in many cases missing values are assigned special characters, such as `-999`, `NA`, `k.A.` etc.; hence, you as a data analyst are responsible for taking appropriate action.    

In [ ]:
powerplants.shape[0]

In [ ]:
powerplants.notnull().sum()

In [ ]:
powerplants.isnull().sum()

In [ ]:
round((powerplants.isnull().sum() / powerplants.shape[0]) * 100, 2)

**Strategies to deal with missing data in Python**

In general there are many options to consider when imputing missing values, for example:
* A constant value that has meaning within the domain, such as 0, distinct from all other values.
* A value from another randomly selected record.
* A mean, median or mode value for the column.
* A value estimated by another predictive model.

There are some libraries implementing more or less advanced missing value imputation strategies such as 

* [`statsmodels`](http://www.statsmodels.org/dev/imputation.html) ([Multiple Imputation with Chained Equations (MICE)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3074241/))
* [`fancyimpute`](https://github.com/iskandr/fancyimpute) (matrix completion and imputation algorithms)
* [`scikit-learn`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) (mean, median, most frequent)
* [`pandas`](https://pandas.pydata.org/pandas-docs/stable/missing_data.html) ([`fillna`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html), [`interpolate`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.interpolate.html) methods)


**Our working strategy to deal with missing data**

_Owing to the fact that the amount of missing values in our data set is considerable high and that we can not easily do predictions or assumptions for the specific missing values, we simply remove these columns with a high amount of missing data._ 

_Nethertheless we have to keep in mind that our dataset still consists of gaps. Therefore we have to handle these gaps individually in the following analysis._

In [ ]:
powerplants

> **Challenge:** Drop all columns in the dataframe that have more than 60 percent null values. Refactor your code afterwards and externalise your code into a function that takes the original dataframe and a threshold. The function should return the new dataframe.

> **Hint**: Make use of the `df.index` and the `df.drop` method.

In [ ]:
# %load ../src/_solutions/filter_dataframe.py

def filter_dataframe(df, threshold):
    p_null = round((df.isnull().sum() / df.shape[0]) * 100, 2)
    return df.drop(p_null[p_null > threshold].index, axis = 1)

In [ ]:
powerplants = filter_dataframe(powerplants, 60)
powerplants

## Dealing with data structures

If we have a closer look on the start date column, we can see that these information is stored as a floating point variable. We can use an information like this for futher proposes. Therefore we only want to extract the year information out of this column and keep only this information.

In [ ]:
powerplants["start date"] = powerplants["start date"].fillna(0.0).astype(int)

In [ ]:
powerplants[["country code", "start date"]].sample(10)

## Visualisation of spatial datasets

_Note: In the subsequent cells we load Python library for spatial data analysis, such as `shapely`, `fiona`, `geopandas`, `cartopy` and `folium`. Make sure that you have installed the [GDAL bindings](http://www.gdal.org/index.html) on your computer._

In this section we make use third party libraries for visualisation of the geospatial information, such as [GeoPandas](http://geopandas.org/index.html) and [shapely](http://toblerity.org/shapely/), which abstract away many algorithmic or computational issues related to spatial data processing and plotting by integrating the workhorses of geospatial computing, such as [GEOS](http://trac.osgeo.org/geos/), [GDAL](http://www.gdal.org/), [OGR](http://gdal.org/1.11/ogr/) and [proj.4](http://proj4.org/), among others.

The geographical information is stored and given by the `latitude` and `longitude` column. We can use these information to localize each power plant in the world.

**Transform the variables `Target Latitude` and `Target Longitude` to spatial coordinates**

In [ ]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(powerplants['longitude'], powerplants['latitude'])]
geometry[0:5]

**Use the GeoPandas to make a pandas `DataFrame` spatially aware.**

[GeoPandas](http://geopandas.org/index.html) extends the datatypes used by pandas to allow spatial operations on geometric types. Geometric operations are performed by [shapely](http://toblerity.org/shapely/). GeoPandas further depends on [fiona](http://toblerity.org/fiona/README.html) for file access and descartes and [matplotlib](https://matplotlib.org/) for plotting.

It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. 

In [ ]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(powerplants, geometry=geometry)
gdf.head()

**Make sure that for every entry we have a valid spatial coordinates**

In [ ]:
print(gdf.shape)
# subset only vaild spatial coordinates
gdf = gdf.loc[gdf[['longitude', 'latitude']].notnull().all(axis = 1)]
print(gdf.shape)
gdf[['longitude', 'latitude']].isnull().sum()

**Assign a spatial coordinate reference system (`crs`) to our GeoPandas object**

In general the CRS may be defined in several ways, for example the CRS may be defined as [Well-known text (WKT)](https://en.wikipedia.org/wiki/Well-known_text) format, or [JSON](https://en.wikipedia.org/wiki/JSON) format, or [GML](https://en.wikipedia.org/wiki/Geography_Markup_Language) format, or in the [Proj4](https://en.wikipedia.org/wiki/PROJ.4) format, among many others.

The Proj4 format is a generic, string-based description of a CRS. It defines projection types and parameter values for particular projections. For instance the Proj4 format string for the [European Terrestrial Reference System 1989 (ETRS89)](https://en.wikipedia.org/wiki/European_Terrestrial_Reference_System_1989) is:

    +proj=longlat +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +no\_defs

With respect to the enormous amount of existing CRS the [International Association of Oil & Gas Producers (IOGP)](https://en.wikipedia.org/wiki/International_Association_of_Oil_%26_Gas_Producers), formerly known as **_European Petroleum Survey Group (EPSG)_**, built a collection of definitions for global, regional, national and local coordinate reference systems and coordinate transformations, the [EPSG Geodetic Parameter Dataset](http://www.epsg.org/). Within this collection each particular coordinate reference systems gets an unique integer identifier, commonly denoted as EPSG. For instance, the EPSG identifier for the the latest revision of the [World Geodetic System (WGS84)](https://en.wikipedia.org/wiki/World_Geodetic_System) is simply [4326](http://spatialreference.org/ref/epsg/4326/).


A nice look up page for different coordinate reference systems is found [here](https://epsg.io/) and a fancy visualization of many prominent map projections is found [here](https://bl.ocks.org/mbostock/raw/3711652/).


In [ ]:
gdf.set_crs('epsg:4326', inplace=True)

### Context matters: Load _Natural Earth countries_ dataset, bundled with GeoPandas

[Natural Earth](http://www.naturalearthdata.com/) is a public domain map dataset available at 1:10m, 1:50m, and 1:110 million scales. Featuring tightly integrated vector and raster data, with Natural Earth you can make a variety of visually pleasing, well-crafted maps with cartography or GIS software. A subset comes bundled with GeoPandas and is accessible from the `gpd.datasets` module. We’ll use it as a helpful global base layer map.



In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head(2)

In [ ]:
world.crs

In [ ]:
world.plot(facecolor='lightgray')

**Combine world map and the power plant data set**

In [ ]:
base = world.plot(facecolor='lightgray')
gdf.plot(ax=base, marker='o', color='red', markersize=10, alpha=0.5);

### Spatial Filtering the data by geolocation

In a world view the points observations overlap each over a lot. So we can not see any pattern. For those purpose it is usefull, to set the focus of the research to a specific geographical region. In the following we want to focus on a continental and a country view.

**Europe focus**

As a first example we want to restrict our analysis to data points, which refer to an area within Europe. Although it is not straightforward to define Europe as an entity, in terms of geography, politics or sphere of cultural identity, we define Europe as an area between the coordinates  

$$\text{33.0 to 73.5°N and 27.0°W to 45.0°E.}$$

In order to represent that area spatially; we construct a `Polygon` object to represent the [bounding box](https://en.wikipedia.org/wiki/Minimum_bounding_box) of Europe. 

In [ ]:
from shapely.geometry import Polygon
# generate geopandas object
poly_europe = gpd.GeoSeries([Polygon([(-27,33), (45,33), (45,73.5), (-27,73.5)])])
bb_europe = gpd.GeoDataFrame({'geometry': poly_europe})
# assign crs
bb_europe.set_crs(epsg=4326, inplace=True)
bb_europe

**Plot world map and bounding box of Europe**

In [ ]:
base = world.plot(facecolor='lightgray')
bb_europe.plot(ax=base, alpha=0.5);

**Subset (intersect) the GeoPandas `DataFrame` with the bounding box of Europe**

In [ ]:
gdf_europe = gpd.sjoin(gdf, bb_europe, how="inner", predicate='intersects').drop("index_right", axis=1)

print(gdf_europe.shape)

In [ ]:
gdf_europe.sample(5)

In [ ]:
gdf_europe.plot();

**In order to keep the spatial context we extract the area of Europe from the world map**

In [ ]:
europe = gpd.overlay(world, bb_europe, how='intersection')
europe.set_crs('epsg:4326', inplace=True)

In [ ]:
base = europe.plot(facecolor='lightgray')
gdf_europe.plot(ax=base, marker='o', color='red', markersize=1.5, alpha=0.75);

**Regional focus - Germany**

In [ ]:
filtered = world[world.name == "Germany"]

In [ ]:
filtered.boundary

In [ ]:
gdf_germany = gpd.sjoin(gdf, filtered, how="inner", predicate='intersects').drop("index_right", axis=1)

In [ ]:
germany = gpd.overlay(world, filtered, how='intersection')
germany.set_crs('epsg:4326', inplace=True)

In [ ]:
base = germany.plot(facecolor='lightgray')
gdf_germany.plot(ax=base, marker='o', color='indigo', markersize=2);

## Saving our Dataframe to disk for furhter analysis

We spend quite a lot of time for prepating our dataset for our analysis. At the end we want to save our dataframe to disk so that we can load this work state at every time again. There exist plenty of options, in this tutorial we want to use the `pickle` library to serialize our dataframe:

In [ ]:
## uncomment to serialize the data to disk
import pickle
pickle.dump(gdf, open("../data/gdf.p", "wb"))

## Ready!! Now it's your turn :)